# 3a. Enrichment Table
  --------------------------------------------------------------------

Create enrichment table with socioeconomic data. This demonstrates a simple way of using V3IO Key-value (KV) to store lookup data.

The data is stored to `/User/examples/model-deployment-with-streaming/data/enrichment-table`. You can create similar enrichments using the same method.

In [1]:
%run config.py

In [2]:
import json
from random import randint
import v3io.dataplane

### Create V3IO Client

In [3]:
v3io_client = v3io.dataplane.Client(endpoint=WEB_API, access_key=V3IO_ACCESS_KEY)

### Create enrichment table
____________________________
We will create enrichment table where the key is postal-code and the value is the socioeconomic index at the area.


In [4]:
for postcode in range(10000,99999):
    remainder = postcode % 3
    if remainder == 0:
        idx = randint(3,5)
    elif remainder == 1:
        idx = randint(1,3)
    else:
        idx = randint(5,7)

    attr = {'postcode': postcode ,'socioeconomic_idx': idx}
    v3io_client.put_item(container=CONTAINER, path=path.join(ENRICHMENT_TABLE_PATH, str(postcode)),
                         attributes=attr)